In [1]:
# 20_newsgroup

"""
1. Dataset passed in must be a CSV file that can be opened using pd.read_csv()
2. perc determines how much of the dataset is used
3. num_topics_word is set to 500 top words
4. num_topics_list is set to [100] but can be changed or appended to, to get more results
5. embedding_model is set to paraphrase-distilroberta-base-v2 (but can be changed to other embedding models)
6. num_epochs_ctm is set to 25 epochs
7. text_column, label_column, and id_column must all be set according to your database
  7.1 text_column is the actual text
  7.2 label_column is what you regress over / predict
  7.3 id_column is the unique identifier for each message
8. dataset_path must be set to where the dataset is stored
9. the name of the folder (be sure to include '/' at the end) -- usually the name of the dataset (i.e. twitter, amazon, etc.)
10. logit should be set to True if target variable is discrete and False if continous
"""

import warnings

warnings.filterwarnings('ignore')

import os
import errno
import nltk
import pandas as pd
from nltk.corpus import stopwords as stop_words
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import string
from sentence_transformers import SentenceTransformer
import scipy.sparse
from contextualized_topic_models.datasets.dataset import CTMDataset
from sklearn.preprocessing import OneHotEncoder
from contextualized_topic_models.models.ctm import CombinedTM
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import KeyedVectors
import gensim.downloader as api
from scipy.spatial.distance import cosine
from gensim.utils import deaccent
import abc
import re
import itertools
import pickle

is_multi = False
is_combined = not is_multi

nltk.download('stopwords')

language = 'en'
stopwords = list(stop_words.words('english'))

num_topics_word = 500
num_topics_list = [100]
embedding_model = 'paraphrase-distilroberta-base-v2'
num_epochs_ctm = 25

# # change these accordingly
text_column = 'message'
label_column = 'label'
id_column = 'Unnamed: 0'
dataset_path = '20_newsgroup.csv'
folder = '20_newsgroup/'

dataset = pd.read_csv(dataset_path)[[id_column, text_column, label_column]].rename(columns = {id_column: 'message_id', text_column: 'message', label_column: 'label'}).dropna()

# shuffles to avoid biases in data
arr = sklearn.utils.shuffle(np.arange(len(dataset)), random_state=42)
dataset = dataset.iloc[arr].reset_index(drop=True)

train, test, message, test_message = train_test_split(dataset, dataset['message'], test_size = 0.20, random_state = 42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
message.reset_index(drop=True, inplace=True)
test_message.reset_index(drop=True, inplace=True)

message = list(message)
test_message = list(test_message)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

class WhiteSpacePreprocessingStopwords():
    """
    Provides a very simple preprocessing script that filters infrequent tokens from text
    """

    def __init__(self, documents, stopwords_list=None, vocabulary_size=2000, max_df=1.0, min_words=1,
                remove_numbers=True):
        """

        :param documents: list of strings
        :param stopwords_list: list of the stopwords to remove
        :param vocabulary_size: the number of most frequent words to include in the documents. Infrequent words will be discarded from the list of preprocessed documents
        :param max_df : float or int, default=1.0
        When building the vocabulary ignore terms that have a document
        frequency strictly higher than the given threshold (corpus-specific
        stop words).
        If float in range [0.0, 1.0], the parameter represents a proportion of
        documents, integer absolute counts.
        This parameter is ignored if vocabulary is not None.
        :param min_words: int, default=1. Documents with less words than the parameter
        will be removed
        :param remove_numbers: bool, default=True. If true, numbers are removed from docs
        """
        self.documents = documents
        if stopwords_list is not None:
            self.stopwords = set(stopwords_list)
        else:
            self.stopwords = []

        self.vocabulary_size = vocabulary_size
        self.max_df = max_df
        self.min_words = min_words
        self.remove_numbers = remove_numbers

    def preprocess(self):
        """
        Note that if after filtering some documents do not contain words we remove them. That is why we return also the
        list of unpreprocessed documents.

        :return: preprocessed documents, unpreprocessed documents and the vocabulary list
        """
        preprocessed_docs_tmp = self.documents
        preprocessed_docs_tmp = [deaccent(doc.lower()) for doc in preprocessed_docs_tmp]
        preprocessed_docs_tmp = [doc.translate(
            str.maketrans(string.punctuation, ' ' * len(string.punctuation))) for doc in preprocessed_docs_tmp]
        if self.remove_numbers:
            preprocessed_docs_tmp = [doc.translate(str.maketrans("0123456789", ' ' * len("0123456789")))
                                    for doc in preprocessed_docs_tmp]
        preprocessed_docs_tmp = [' '.join([w for w in doc.split() if len(w) > 0 and w not in self.stopwords])
                                for doc in preprocessed_docs_tmp]

        vectorizer = CountVectorizer(max_features=self.vocabulary_size, max_df=self.max_df)
        vectorizer.fit_transform(preprocessed_docs_tmp)
        temp_vocabulary = set(vectorizer.get_feature_names_out())

        preprocessed_docs_tmp = [' '.join([w for w in doc.split() if w in temp_vocabulary])
                                for doc in preprocessed_docs_tmp]

        preprocessed_docs, unpreprocessed_docs, retained_indices = [], [], []
        for i, doc in enumerate(preprocessed_docs_tmp):
            if len(doc) > 0 and len(doc) >= self.min_words:
                preprocessed_docs.append(doc)
                unpreprocessed_docs.append(self.documents[i])
                retained_indices.append(i)

        vocabulary = list(set([item for doc in preprocessed_docs for item in doc.split()]))

        return preprocessed_docs, unpreprocessed_docs, vocabulary, retained_indices

def get_bag_of_words(data, min_length):
    """
    Creates the bag of words
    """
    vect = [np.bincount(x[x != np.array(None)].astype('int'), minlength=min_length)
            for x in data if np.sum(x[x != np.array(None)]) != 0]

    vect = scipy.sparse.csr_matrix(vect)
    return vect


def bert_embeddings_from_file(text_file, sbert_model_to_load, batch_size=200, max_seq_length=None):
    """
    Creates SBERT Embeddings from an input file, assumes one document per line
    """

    model = SentenceTransformer(sbert_model_to_load)

    if max_seq_length is not None:
        model.max_seq_length = max_seq_length

    with open(text_file, encoding="utf-8") as filino:
        texts = list(map(lambda x: x, filino.readlines()))

    check_max_local_length(max_seq_length, texts)

    return np.array(model.encode(texts, show_progress_bar=True, batch_size=batch_size))


def bert_embeddings_from_list(texts, sbert_model_to_load, batch_size=200, max_seq_length=None):
    """
    Creates SBERT Embeddings from a list
    """
    model = SentenceTransformer(sbert_model_to_load)

    if max_seq_length is not None:
        model.max_seq_length = max_seq_length

    check_max_local_length(max_seq_length, texts)

    return np.array(model.encode(texts, show_progress_bar=True, batch_size=batch_size))


def check_max_local_length(max_seq_length, texts):
    max_local_length = np.max([len(t.split()) for t in texts])
    if max_local_length > max_seq_length:
        warnings.simplefilter('always', DeprecationWarning)


class TopicModelDataPreparation:

    def __init__(self, contextualized_model=None, show_warning=True, max_seq_length=128):
        self.contextualized_model = contextualized_model
        self.vocab = []
        self.id2token = {}
        self.vectorizer = None
        self.label_encoder = None
        self.show_warning = show_warning
        self.max_seq_length = max_seq_length

    def load(self, contextualized_embeddings, bow_embeddings, id2token, labels=None):
        return CTMDataset(
            X_contextual=contextualized_embeddings, X_bow=bow_embeddings, idx2token=id2token, labels=labels)

    def fit(self, text_for_contextual, text_for_bow, labels=None, custom_embeddings=None):
        """
        This method fits the vectorizer and gets the embeddings from the contextual model

        :param text_for_contextual: list of unpreprocessed documents to generate the contextualized embeddings
        :param text_for_bow: list of preprocessed documents for creating the bag-of-words
        :param custom_embeddings: np.ndarray type object to use custom embeddings (optional).
        :param labels: list of labels associated with each document (optional).
        """

        if custom_embeddings is not None:
            assert len(text_for_contextual) == len(custom_embeddings)

            if text_for_bow is not None:
                assert len(custom_embeddings) == len(text_for_bow)

            if type(custom_embeddings).__module__ != 'numpy':
                raise TypeError("contextualized_embeddings must be a numpy.ndarray type object")

        if text_for_bow is not None:
            assert len(text_for_contextual) == len(text_for_bow)

        if self.contextualized_model is None and custom_embeddings is None:
            raise Exception("A contextualized model or contextualized embeddings must be defined")

        # TODO: this count vectorizer removes tokens that have len = 1, might be unexpected for the users
        self.vectorizer = CountVectorizer()

        train_bow_embeddings = self.vectorizer.fit_transform(text_for_bow)

        # if the user is passing custom embeddings we don't need to create the embeddings using the model

        if custom_embeddings is None:
            train_contextualized_embeddings = bert_embeddings_from_list(
                text_for_contextual, sbert_model_to_load=self.contextualized_model, max_seq_length=self.max_seq_length)
        else:
            train_contextualized_embeddings = custom_embeddings
        self.vocab = self.vectorizer.get_feature_names_out()
        self.id2token = {k: v for k, v in zip(range(0, len(self.vocab)), self.vocab)}

        if labels:
            self.label_encoder = OneHotEncoder()
            encoded_labels = self.label_encoder.fit_transform(np.array([labels]).reshape(-1, 1))
        else:
            encoded_labels = None
        return CTMDataset(
            X_contextual=train_contextualized_embeddings, X_bow=train_bow_embeddings,
            idx2token=self.id2token, labels=encoded_labels)

    def transform(self, text_for_contextual, text_for_bow=None, custom_embeddings=None, labels=None):
        """
        This method create the input for the prediction. Essentially, it creates the embeddings with the contextualized
        model of choice and with trained vectorizer.

        If text_for_bow is missing, it should be because we are using ZeroShotTM

        :param text_for_contextual: list of unpreprocessed documents to generate the contextualized embeddings
        :param text_for_bow: list of preprocessed documents for creating the bag-of-words
        :param custom_embeddings: np.ndarray type object to use custom embeddings (optional).
        :param labels: list of labels associated with each document (optional).
        """

        if custom_embeddings is not None:
            assert len(text_for_contextual) == len(custom_embeddings)

            if text_for_bow is not None:
                assert len(custom_embeddings) == len(text_for_bow)

        if text_for_bow is not None:
            assert len(text_for_contextual) == len(text_for_bow)

        if self.contextualized_model is None:
            raise Exception("You should define a contextualized model if you want to create the embeddings")

        if text_for_bow is not None:
            test_bow_embeddings = self.vectorizer.transform(text_for_bow)
        else:
            # dummy matrix
            if self.show_warning:
                warnings.simplefilter('always', DeprecationWarning)
                warnings.warn(
                    "The method did not have in input the text_for_bow parameter. This IS EXPECTED if you "
                    "are using ZeroShotTM in a cross-lingual setting")

            # we just need an object that is matrix-like so that pytorch does not complain
            test_bow_embeddings = scipy.sparse.csr_matrix(np.zeros((len(text_for_contextual), 1)))

        if custom_embeddings is None:
            test_contextualized_embeddings = bert_embeddings_from_list(
                text_for_contextual, sbert_model_to_load=self.contextualized_model, max_seq_length=self.max_seq_length)
        else:
            test_contextualized_embeddings = custom_embeddings

        if labels:
            encoded_labels = self.label_encoder.transform(np.array([labels]).reshape(-1, 1))
        else:
            encoded_labels = None

        return CTMDataset(X_contextual=test_contextualized_embeddings, X_bow=test_bow_embeddings,
                        idx2token=self.id2token, labels=encoded_labels)

documents = [line.strip() for line in (message + test_message) if not isinstance(line, float)]
test_documents = [line.strip() for line in test_message if not isinstance(line, float)]

sp_train = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp_train.preprocess()
labels = pd.concat([train, test]).reset_index()['label'][retained_indices]

sp_test = WhiteSpacePreprocessingStopwords(test_documents, stopwords_list=stopwords)
test_preprocessed_documents, test_unpreprocessed_corpus, test_vocab, test_retained_indices = sp_test.preprocess()
test_labels = test['label'][test_retained_indices]

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pickle
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, r2_score

dataset = '20_newsgroup' # folder and dataset name
# cols = ['gender', 'age', 'politics'] # outcome columns
cols = ['label_x'] # outcome columns
outcome = 'label'

num_epochs = 500

df = pd.read_csv(dataset + ".csv")

print(dataset)

all_scores = {}

train = pickle.load(open(dataset + '/BERTopic/run_1/train.pkl', 'rb'))
test = pickle.load(open(dataset + '/BERTopic/run_1/test.pkl', 'rb'))

for model in ['GensimLDA', 'BERTopic', 'NMF', 'Mallet_LDA', 'CTM']:
    print(model)
    
    train_error_list = []
    test_error_list = []

    r2s_train = []
    r2s_test = []

    if model == 'GensimLDA':
        lr = 0.05
    elif model == 'BERTopic':
        lr = 0.05
    elif model == 'NMF':
        lr = 0.003
    elif model == 'Mallet_LDA':
        lr = 0.003
    else:
        lr = 0.003
        df = df.iloc[retained_indices]

    for run in range(1, 6):
        print("Run: " + str(run))

        # loading in distributions that were saved during topic extraction
        test_distribution = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_topic_distribution_test.pkl', 'rb'))
        train_distribution = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_topic_distribution_train.pkl', 'rb'))

        if model == 'CTM':
            train_distribution = train_distribution[:round(len(train_distribution) * 0.80)]
        
        topics = []
        with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_100.txt', 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                topic_list = [item.strip() for item in row if item.strip()]
                topics.append(topic_list)

        temp = pd.concat([train, test]).reset_index(drop=True) # concatenating train and test datasets
        distribution = np.concatenate([train_distribution, test_distribution]) # concatenating train and test distributions

        merged = pd.merge(temp, df, how='inner', left_on = 'message_id', right_on = 'Unnamed: 0')[['message_id', 'message_x'] + cols]
        merged.columns = ['message_id', 'message', 'label']

        X = distribution
        y = merged[outcome].reset_index(drop=True) # the outcome we care about

        # 80-20 split --> didn't use train-test-split function since its already shuffled
        X_train = X[:round(0.80 * len(X))]
        X_test = X[round(0.80 * len(X)):]

        y_train = y[:round(0.80 * len(X))]
        y_test = y[round(0.80 * len(X)):]

        if model == 'CTM':
            X_train = train_distribution
            X_test = test_distribution
            y_train = np.array(labels[:round(len(labels) * 0.80)])
            y_test = np.array(test_labels)

        # Convert arrays to torch tensors
        X_train_tensor = torch.tensor(np.array(X_train).astype(np.float32))
        y_train_tensor = torch.tensor(np.array(y_train).astype(np.longlong))  # Use long for classification
        X_test_tensor = torch.tensor(np.array(X_test).astype(np.float32))
        y_test_tensor = torch.tensor(np.array(y_test).astype(np.longlong))

        # Create datasets and dataloaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

        train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

        # Define the model
        class LogisticRegressionModel(nn.Module):
            def __init__(self, input_size, num_classes):
                super(LogisticRegressionModel, self).__init__()
                self.layer1 = nn.Linear(input_size, num_classes)

            def forward(self, x):
                return self.layer1(x)

        input_size = X_train.shape[1]
        num_classes = len(np.unique(y_train))  # Assuming y_train contains all classes
        logit_model = LogisticRegressionModel(input_size, num_classes)

        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()  # This includes softmax
        optimizer = optim.Adam(logit_model.parameters(), lr=lr)

        loss_values = []
        early_stopping_triggered = False

        # Train the model
        for epoch in range(num_epochs):
            epoch_loss = 0
            for inputs, targets in train_loader:
                # Forward pass
                outputs = logit_model(inputs)
                loss = criterion(outputs, targets)
                
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item() * inputs.size(0) 
            
            epoch_loss /= len(train_loader.dataset)
            loss_values.append(epoch_loss)

            if epoch >= 50:
                # Calculate the percentage change in loss
                loss_change = (loss_values[epoch - 50] - loss_values[epoch]) / loss_values[epoch - 50]
                
                # If change in loss is less than 1%, stop training
                if abs(loss_change) < 0.01:
                    print(f'Early stopping at epoch {epoch+1} due to minimal loss improvement.')
                    early_stopping_triggered = True
                    break

            if (epoch+1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # Predict on the test set
        logit_model.eval()  # Set the model to evaluation mode

        with torch.no_grad():
            y_pred_train = logit_model(X_train_tensor)
            _, predicted_train = torch.max(y_pred_train.data, 1)
            y_pred_test = logit_model(X_test_tensor)
            _, predicted_test = torch.max(y_pred_test.data, 1)

        train_error = 1 - accuracy_score(predicted_train, y_train)
        test_error = 1 - accuracy_score(predicted_test, y_test)

        train_error_list.append(train_error)
        test_error_list.append(test_error)
        
        print(f'Train Error: {train_error}')
        print(f'Test Error: {test_error}')

        print()

        r2_train = r2_score(y_train, predicted_train)
        r2_test = r2_score(y_test, predicted_test)

        r2s_train.append(r2_train)
        r2s_test.append(r2_test)
        
        print(f'R2 Train: {r2_train}')
        print(f'R2 Test: {r2_test}')

        all_scores[model] = {
            'Train Error': train_error_list,
            'Test Error': test_error_list,
            'R2 Train': r2s_train,
            'R2 Test': r2s_test,

            'y_train': y_train,
            'y_train_pred': predicted_train.numpy(),
            'y_test': y_test,
            'y_test_pred': predicted_test.numpy()
        }
    
    print()

print()

for m in all_scores.keys():
    print(f'{m} Average Train Error: {np.mean(all_scores[m]["Train Error"])}')
    print(f'{m} Average Test Error: {np.mean(all_scores[m]["Test Error"])}')
    print(f'{m} Average R2 Train: {np.mean(all_scores[m]["R2 Train"])}')
    print(f'{m} Average R2 Test: {np.mean(all_scores[m]["R2 Test"])}')
    print()

with open('all_results/' + dataset + '_all_scores_' + outcome + '.pkl', 'wb') as f:
     pickle.dump(all_scores, f)

[nltk_data] Downloading package stopwords to /Users/Arnav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


20_newsgroup
GensimLDA
Run: 1
Epoch [10/500], Loss: 2.8390
Epoch [20/500], Loss: 2.7178
Epoch [30/500], Loss: 2.6204
Epoch [40/500], Loss: 2.5397
Epoch [50/500], Loss: 2.4716
Epoch [60/500], Loss: 2.4132
Epoch [70/500], Loss: 2.3626
Epoch [80/500], Loss: 2.3183
Epoch [90/500], Loss: 2.2793
Epoch [100/500], Loss: 2.2448
Epoch [110/500], Loss: 2.2139
Epoch [120/500], Loss: 2.1862
Epoch [130/500], Loss: 2.1612
Epoch [140/500], Loss: 2.1386
Epoch [150/500], Loss: 2.1181
Epoch [160/500], Loss: 2.0994
Epoch [170/500], Loss: 2.0822
Epoch [180/500], Loss: 2.0665
Epoch [190/500], Loss: 2.0521
Epoch [200/500], Loss: 2.0387
Epoch [210/500], Loss: 2.0264
Epoch [220/500], Loss: 2.0149
Epoch [230/500], Loss: 2.0043
Epoch [240/500], Loss: 1.9945
Epoch [250/500], Loss: 1.9853
Epoch [260/500], Loss: 1.9767
Epoch [270/500], Loss: 1.9686
Epoch [280/500], Loss: 1.9611
Epoch [290/500], Loss: 1.9541
Epoch [300/500], Loss: 1.9474
Epoch [310/500], Loss: 1.9412
Epoch [320/500], Loss: 1.9353
Epoch [330/500], Lo

In [4]:
all_scores

{'GensimLDA': [0.4889589905362776,
  0.4889589905362776,
  0.4889589905362776,
  0.4889589905362776,
  0.4889589905362776],
 'BERTopic': [0.5691753041910771,
  0.5651194231635872,
  0.5718792248760703,
  0.5687246507435781,
  0.5844975214060387],
 'NMF': [0.4353312302839117,
  0.4276701216764308,
  0.423614240648941,
  0.4362325371789094,
  0.43352861649391616],
 'Mallet_LDA': [0.42586750788643535,
  0.41324921135646686,
  0.42496620099143756,
  0.42000901306895,
  0.41685443893645785],
 'CTM': [0.45196706312900276,
  0.47209515096065874,
  0.4830741079597438,
  0.48947849954254347,
  0.4739249771271729]}